In [1]:
import gc
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import seaborn as sns
sns.set(style='whitegrid')
sns.set(rc={'figure.figsize':(17, 9)})

import catboost
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 
from sklearn.metrics import mean_squared_error

RAND = 10

In [2]:
# df_cabin = pd.concat([pd.read_csv('/kaggle/input/cabin-2018/CABIN_2018 (1).csv', sep=';'), pd.read_csv('/kaggle/input/cabin-2019/CABIN_2019.csv', sep=';')]).reset_index(drop= True)
# df_cabin.columns = df_cabin.columns.str.lower()
# df_cabin['dep_date'] = pd.to_datetime(df_cabin['dep_date'])
# # df_cabin['dat_s'] = pd.to_datetime(df_cabin['dat_s'])
# df_cabin['seg_orig'] = df_cabin['seg_orig'].map(str.strip)
# df_cabin['seg_dest'] = df_cabin['seg_dest'].map(str.strip)
# df_cabin['ssc'] = df_cabin['ssc'].map(str.strip)
# df_cabin['cap'] = df_cabin['cap'].map(str.strip)
# df_cabin['ual'] = df_cabin['ual'].map(str.strip)
# df_cabin['ual'] = df_cabin['ual'].map(str.strip)
# df_cabin['equip'] = df_cabin['equip'].map(str.strip)
# df_cabin.drop(['dat_s', 'sak', 'lgnum'], 1, inplace=True)

# df = pd.DataFrame()

# for i_y in range(2018, 2020):
#   for i_m in range(1, 13):
#     if i_m < 10:
#       df_class = pd.read_csv(f'/kaggle/input/data-class/CLASS_0{i_m}{i_y}/CLASS_0{i_m}{i_y}.csv', sep=';')
#     else:
#       df_class = pd.read_csv(f'/kaggle/input/data-class/CLASS_{i_m}{i_y}/CLASS_{i_m}{i_y}.csv', sep=';')

#     df_class.columns = df_class.columns.str.lower()
# #     df_class['sdat_s'] = pd.to_datetime(df_class['sdat_s'])
#     df_class['dd'] = pd.to_datetime(df_class['dd'])

#     df_class.drop(['sdat_s', 'sak', 'seg_num'], 1, inplace=True)

#     df_class['sorg'] = df_class['sorg'].map(str.strip)
#     df_class['sdst'] = df_class['sdst'].map(str.strip)
#     df_class['sscl1'] = df_class['sscl1'].map(str.strip)
#     df_class['seg_class_code'] = df_class['seg_class_code'].map(str.strip)
#     df_class['nbcl'] = df_class['nbcl'].map(str.strip)

#     df0 = pd.merge(df_class, df_cabin, left_on=['flt_num', 'dd', 'sorg', 'sdst', 'dtd'], right_on=['flt_num', 'dep_date', 'seg_orig', 'seg_dest', 'dtd'])

#     df0['weekday'] = [m.dayofweek for m in df0['dd']]
#     df0['month'] = [m.month for m in df0['dd']]
#     df0['year'] = [m.year for m in df0['dd']]
#     # df0['dtd'] = [dtd//10 for dtd in df0['dtd']]
#     df0['tt_dep'] = [tt//400 for tt in df0['tt_dep']]
#     df0['tt_arr'] = [tt//400 for tt in df0['tt_arr']]

#     df0.drop(['seg_orig', 'ns', 'fclcld', 'ssc', 'seg_dest', 'dep_date', 'nbcl', 'cap', 'ual', 'sal', 'dd', 'sa', 'au'], 1, inplace=True)
#     df0 = df0.astype({'flt_num': np.uint16, 'pass_bk':np.uint16, 'pass_dep': np.uint16, 'dtd': np.int16, 'tt_dep': np.uint8, 'tt_arr': np.uint8, 'weekday': np.uint8, 'month': np.uint8, 'year': np.uint16})

#     df = pd.concat([df, df0])

# compression_opts = dict(method='zip',
#                         archive_name='df.csv')  


# df.to_csv('df.zip', index=False, compression=compression_opts)

In [2]:
df = pd.read_csv(open(r'C:\Users\Dima\Downloads\Telegram Desktop\df (1)\df.csv'))
df_ml_pass_dep = df[df['dtd'] == -1].drop(['pass_bk', 'dtd'], axis=1)
df = df.drop(['pass_dep'], axis=1)

In [12]:
y, X = df_ml_pass_dep.pass_dep, df_ml_pass_dep.drop(['year', 'flt_num', 'pass_dep'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RAND)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RAND)

cat_features = ['equip', 'sscl1', 'seg_class_code', 'sdst', 'sorg']
# 'weekday', 'month', 'tt_dep', 'tt_arr',
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
validation_pool = Pool(data=X_val, label=y_val, cat_features=cat_features)

model_pass_dep = CatBoostRegressor(
    iterations=50000, 
    random_seed=RAND, 
    task_type='GPU',
    learning_rate=0.01,
    use_best_model=True,
    early_stopping_rounds=500,
    depth=9)
model_pass_dep.fit(
    train_pool, 
    eval_set=validation_pool,
    verbose=5000)

0:	learn: 13.7552445	test: 13.6732979	best: 13.6732979 (0)	total: 43.6ms	remaining: 36m 18s
5000:	learn: 8.0367991	test: 8.4195404	best: 8.4195219 (4999)	total: 3m 23s	remaining: 30m 31s
10000:	learn: 7.6513817	test: 8.3736713	best: 8.3734789 (9813)	total: 7m 11s	remaining: 28m 44s
bestTest = 8.373478861
bestIteration = 9813
Shrink model to first 9814 iterations.


In [13]:
model_pass_dep.best_score_

{'learn': {'RMSE': 7.631597067151796},
 'validation': {'RMSE': 8.37347886105613}}

In [14]:
model_pass_dep.best_iteration_

9813

In [15]:
model_pass_dep.get_feature_importance(prettified=True)

,Feature Id,Importances
0,seg_class_code,56.134148
1,month,12.340629
2,equip,9.114389
3,weekday,5.313152
4,tt_arr,3.661820
5,tt_dep,3.646850
6,sscl1,3.537700
7,sorg,3.379240
8,sdst,2.872073


In [16]:
raw_pred = model_pass_dep.predict(X_test)

In [17]:
r2_score(y_test, raw_pred)

0.6271263187515241

In [18]:
mean_squared_error(y_test, raw_pred)

69.06154250544512

In [10]:
model_pass_dep.save_model('model_pass_dep.cbm', format='cbm')

In [68]:
df.columns

Index(['flt_num', 'sorg', 'sdst', 'sscl1', 'seg_class_code', 'pass_bk', 'dtd',
       'tt_dep', 'tt_arr', 'equip', 'weekday', 'month', 'year'],
      dtype='object')

In [4]:
df = df.groupby(['flt_num', 'sorg', 'sdst', 'sscl1', 'pass_bk', 'dtd', 'tt_dep', 'tt_arr', 'equip', 'weekday', 'month', 'year']).agg(pass_bk_sum = ('pass_bk', 'sum')).reset_index()

In [5]:
df = df.drop(['pass_bk'], axis=1)

In [6]:
df.columns

Index(['flt_num', 'sorg', 'sdst', 'sscl1', 'dtd', 'tt_dep', 'tt_arr', 'equip',
       'weekday', 'month', 'year', 'pass_bk_sum'],
      dtype='object')

In [7]:
df_ml_pass_dep.columns

Index(['flt_num', 'sorg', 'sdst', 'sscl1', 'seg_class_code', 'pass_dep',
       'tt_dep', 'tt_arr', 'equip', 'weekday', 'month', 'year'],
      dtype='object')

In [8]:
# df_ml_pass_dep = df_ml_pass_dep.drop(['seg_class_code'], axis=1)
# df = pd.merge(df, df_ml_pass_dep, on=['flt_num', 'sorg', 'sdst', 'sscl1', 'tt_dep', 'tt_arr', 'equip', 'weekday', 'month', 'year'])

In [9]:
y, X = df.pass_bk_sum, df.drop(['year', 'flt_num', 'pass_bk_sum'], axis=1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RAND)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RAND)

In [11]:
cat_features = ['equip', 'sdst', 'sorg', 'sscl1']
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
validation_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)

In [47]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7263633 entries, 7352445 to 2519454
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   sorg     object
 1   sdst     object
 2   dtd      int64 
 3   tt_dep   int64 
 4   tt_arr   int64 
 5   equip    object
 6   weekday  int64 
 7   month    int64 
dtypes: int64(5), object(3)
memory usage: 498.8+ MB


In [12]:
model = CatBoostRegressor(
    iterations=5000, 
    task_type='GPU',
    random_seed=RAND, 
    learning_rate=0.05, 
    use_best_model=True,
    early_stopping_rounds=50,
    depth=7)
model.fit(
    train_pool, 
    eval_set=validation_pool,
    verbose=500
)

0:	learn: 11.6781650	test: 11.6861698	best: 11.6861698 (0)	total: 754ms	remaining: 1h 2m 49s
500:	learn: 10.3383688	test: 10.3477685	best: 10.3477685 (500)	total: 5m 25s	remaining: 48m 40s
1000:	learn: 10.3145808	test: 10.3261313	best: 10.3261303 (999)	total: 10m 42s	remaining: 42m 46s
1500:	learn: 10.3015939	test: 10.3149676	best: 10.3149661 (1499)	total: 16m 8s	remaining: 37m 38s
2000:	learn: 10.2926389	test: 10.3076877	best: 10.3076877 (2000)	total: 21m 48s	remaining: 32m 41s
2500:	learn: 10.2857320	test: 10.3024638	best: 10.3024589 (2498)	total: 27m 37s	remaining: 27m 36s
3000:	learn: 10.2802127	test: 10.2985869	best: 10.2985869 (3000)	total: 33m 30s	remaining: 22m 19s
3500:	learn: 10.2752198	test: 10.2952551	best: 10.2952541 (3499)	total: 39m 19s	remaining: 16m 50s
4000:	learn: 10.2709767	test: 10.2926053	best: 10.2926053 (4000)	total: 45m 1s	remaining: 11m 14s
4500:	learn: 10.2672886	test: 10.2905377	best: 10.2905377 (4500)	total: 50m 48s	remaining: 5m 38s
4999:	learn: 10.2640789

In [15]:
model.best_iteration_

4995

In [16]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,sscl1,43.999299
1,dtd,17.041089
2,equip,11.015320
3,month,9.445063
4,sorg,4.384703
5,tt_dep,4.089823
6,weekday,3.906906
7,tt_arr,3.746849
8,sdst,2.370948


In [17]:
raw_pred = model.predict(
  X_test
)

In [18]:
r2_score(y_test, raw_pred)

0.2394877397827675

In [19]:
mean_squared_error(y_test, raw_pred)

105.85889265250422

In [20]:
model.save_model('model.cbm', format='cbm')

In [21]:
X_test.columns

Index(['sorg', 'sdst', 'sscl1', 'dtd', 'tt_dep', 'tt_arr', 'equip', 'weekday',
       'month'],
      dtype='object')

In [23]:
X_test.sscl1.unique()

array(['Y', 'C'], dtype=object)

In [25]:
X_test.dtd.max()

285